In [ ]:
import os
import json
from gcs_manager import GCSManager

def main():
    """Runs the GCP Storage demonstration based on a JSON plan."""
    print("--- Starting GCP Bucket Demonstration ---")
    
    # --- Configuration ---
    credentials_path = os.path.join('env', 'test-team-463805-c88698eab812.json')
    project_id = "test-team-463805"
    bucket_name = f"{project_id}-demo-bucket-aip"
    demo_plan_path = "demo_plan.json"
    local_file_to_upload = "demo_upload.txt"
    downloaded_file_name = "demo_downloaded.txt" # Needed for cleanup

    # --- Setup ---
    print("\n--- Preparing local files for Demo ---")
    with open(local_file_to_upload, "w") as f:
        f.write("Hello, Google Cloud Storage!")
    print(f"Created local file: '{local_file_to_upload}'")
    print("-" * 30)

    try:
        # --- Initialization ---
        gcs_manager = GCSManager(credentials_path, bucket_name)

        # --- Load and Run Demonstration Plan ---
        print(f"--- Loading demonstration plan from '{demo_plan_path}' ---")
        with open(demo_plan_path, 'r') as f:
            plan = json.load(f)
        
        print("--- Executing demonstration plan ---")
        for i, step in enumerate(plan):
            action = step.get("action")
            params = step.get("params", {})
            
            if not action:
                print(f"Skipping step {i+1}: No action defined.")
                continue

            try:
                method_to_call = getattr(gcs_manager, action)
                method_to_call(**params)
            except AttributeError:
                print(f"Error in step {i+1}: Action '{action}' not found in GCSManager.")
            except Exception as e:
                print(f"An error occurred during action '{action}': {e}")

        # --- Optional Cleanup on GCP ---
        # Uncomment the following line to delete the bucket and all its contents
        # gcs_manager.delete_bucket()

    except Exception as e:
        print(f"A critical error occurred: {e}")
    finally:
        # --- Local Cleanup ---
        print("\n--- Cleaning up local files ---")
        if os.path.exists(local_file_to_upload):
            os.remove(local_file_to_upload)
        if os.path.exists(downloaded_file_name):
            os.remove(downloaded_file_name)
        print("Local files removed.")
        print("-" * 30)
        print("Demonstration complete.")

if __name__ == "__main__":
    main()
